In [12]:
import numpy as np

from pydrake.common.value import Value
from pydrake.geometry import StartMeshcat, MeshcatVisualizer
from pydrake.systems.framework import DiagramBuilder, LeafSystem, BasicVector
from pydrake.multibody.math import SpatialForce
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph, ExternallyAppliedSpatialForce, ExternallyAppliedSpatialForceMultiplexer
from pydrake.multibody.parsing import Parser
from pydrake.all import Simulator


# pydot = interface to graphViz,consist in a parser for the DotFormat used by GraphViz
from pydot import graph_from_dot_data

In [2]:
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7000


In [61]:
class Controller(LeafSystem):

    def __init__(self): 
        LeafSystem.__init__(self)
        #self.DeclareVectorInputPort("u", BasicVector(2)) 
        self.DeclareVectorOutputPort("y", BasicVector(2), self.CalcOutputY)
        self.t = 50
    def CalcOutputY(self, context, output): 
        q1dotdot = 0
        q2dotdot = 0
        output.SetFromVector([q1dotdot, q2dotdot])
    

class CartesianForceEE_Sys(LeafSystem):
    def __init__(self, end_effector_id):
        LeafSystem.__init__(self)
        self.force_obj = ExternallyAppliedSpatialForce()
        self.force_obj.body_index = end_effector_id
        force_cls = Value[list[ExternallyAppliedSpatialForce]]
        self.DeclareAbstractOutputPort("F-cartesian", 
                                       lambda: force_cls(), 
                                       self.CalcSpatialForce)

    def CalcSpatialForce(self, context, applied_force):
        F = SpatialForce(tau=np.zeros(3), f=np.array([1., 0., 0.]))
        self.force_obj.F_Bq_W = F
        applied_force.set_value([self.force_obj])


In [62]:
builder = DiagramBuilder()

time_step = 0.003
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step)
controller = builder.AddNamedSystem("controller",Controller())
Parser(plant, scene_graph).AddModels("../urdf_model/RR_planar.sdf")
plant.Finalize()
builder.Connect(controller.get_output_port(), plant.get_actuation_input_port())

In [63]:

end_effector_id = plant.GetBodyByName('end-effector').index()
force_sys = CartesianForceEE_Sys(end_effector_id)
builder.AddSystem(force_sys)
builder.Connect(force_sys.get_output_port(0), plant.get_applied_spatial_force_input_port())

In [64]:
visualizer = MeshcatVisualizer.AddToBuilder(
    builder, 
    scene_graph.get_query_output_port(),
    meshcat
)


In [65]:

diagram = builder.Build()
diagram_root_context = diagram.CreateDefaultContext()
    

In [66]:
plant_context = plant.GetMyContextFromRoot(diagram_root_context)

plant.GetJointByName('q1').set_angle(plant_context, -np.pi)
plant.GetJointByName('q2').set_angle(plant_context, np.pi/2)

#plant.get_actuation_input_port().FixValue(plant_context, np.ones(2))
#print(plant_context)
diagram.ForcedPublish(diagram_root_context)


In [68]:
simulator = Simulator(diagram, diagram_root_context)
simulator.AdvanceTo(50)
print(plant_context)

RuntimeError: C++ object must be owned by pybind11 when attempting to release to C++

In [12]:
def save_plant_svg_graph(diagram, destination_path):
    dot_format_graph = graph_from_dot_data(diagram.GetGraphvizString())[0]
    binary_data_svg = dot_format_graph.create_svg()
    with open(destination_path, 'wb') as f:
        f.write(binary_data_svg)

save_plant_svg_graph(diagram, destination_path="../log/RR_planar.svg")

In [ ]:
from pydrake.all import AbstractValue

class EE_force_applier(LeafSystem):

    def __init__(self, end_effector_id):
        LeafSystem.__init__(self)
        self.external_force_obj = ExternallyAppliedSpatialForce()
        self.external_force_obj.body_index = end_effector_id
        self.DeclareAbstractOutputPort('force', AbstractValue.Make(self.external_force_obj), self.CalcSpatialForce)
        
    def CalcSpatialForce(self, context, output):
        f_x = 0.5
        f_y = 0.5
        self.external_force_obj.F_Bq_w = SpatialForce(tau=np.zeros(3), f=np.array([1.3, 1.5, 0]))
        
